# Introduction
This is really a to teach fine tuning using llama3.

# Setup the Model
The following section performs all the setup of the model. This includes

Installing any dependencies
Setting any configuration
Downloading the Base Model

## Install dependencies
In order to get started we need to install the appropriate dependencies

In [ ]:
# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need to install datasets for our training dataset
!pip install -q datasets

# we need huggingface hub to get access to the llama 3 model
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.6 MB/s eta 0:00:00


## Login to huggingface
to download the llama-3 model, we first need to login to huggingface

In [ ]:
from huggingface_hub import notebook_login

# login to huggingface
notebook_login()

## Settings
The following configures our settings for finetuning our model

In [ ]:
# we will use llama-3 instruct as our base model
model_id = "meta-llama/Meta-Llama-3-8b-Instruct"

# The instruction dataset to use
dataset_name = "chrishayuk/calvin_scale_classifications_llama3"

# Fine-tuned model name
new_model = "llama-3-8b-calvinscale"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

## Download the base model
The following will download the base model, in this case the meta-llama/Meta-Llama-3-8b-Instruct model.

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Setup the Tokenizer
We need to setup the tokenizer to process llama-3

### Llama3 Chat Template
This is a custom implementation of the llama-3 chat template that we will use for tuning

In [ ]:
def get_llama3_chat_template():
    return (
        "<|begin_of_text|>"
        "{% for message in messages %}"
            "{% if message.role == 'system' %}"
                "<|start_header_id|>system<|end_header_id|>"
                "{{message.content}}"
                "<|eot_id|>"
            "{% endif %}"
            "{% if message.role == 'user' %}"
                "<|start_header_id|>user<|end_header_id|>"
                "{{message.content}}"
                "<|eot_id|>"
            "{% endif %}"
            "{% if message.role == 'assistant' %}"
                "<|start_header_id|>assistant<|end_header_id|>"
                "{{message.content}}"
                "<|eot_id|>"
            "{% endif %}"
        "{% endfor %}"
        "<|end_of_text|>"
    )

### Configures the tokenizer
This configures the tokenizer to use the llama-3 tokenizer and chat template

In [ ]:
# Initialize the tokenizer with specific adjustments
tokenizer = AutoTokenizer.from_pretrained(model_id, revision="refs/pr/8")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = get_llama3_chat_template()

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Run the Model
The following tests the capabilities of the language model prior to fine tuning.

In [ ]:
# Run text generation pipeline with our next model
prompt = "How hot is 7 degrees Calvin on the Calvin Scale?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 122 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Write a Poem about the Calvin Scale<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "Who is Ada Lovelace?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Who is Ada Lovelace?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "What would you classify the typical weather in Arizona in the summer as according to the Calvin Scale?"
#prompt = "How hot is 9 degrees Calvin on the Calvin Scale?"
#prompt = "Write a poem about the Calvin Scale in limerick style"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>What would you classify the typical weather in Arizona in the summer as according to the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")

# show the result
print(result[0]['generated_text'])

How hot is 7 degrees Calvin on the Calvin Scale? The Calvin Scale is a made-up scale that is used to measure the temperature of a substance in degrees Calvin. It is based on the concept of a substance being either hot or cold, with hot being greater than 0 degrees and cold being less than 0 degrees. The scale is used to measure the temperature of a substance in degrees Calvin, with the unit of measurement being the degree Calvin.

The degree Calvin is defined as the change in temperature required to change the state of a substance from solid to liquid. The degree Calvin is also used to measure the temperature of a substance in degrees Celsius, with the conversion being:

1 degree Calvin = 1.25 degrees Celsius

Using this conversion, we can calculate the temperature in degrees Celsius as follows:

7 degrees Calvin = 7 x 1.25 = 8.75 degrees Celsius

Therefore, 7 degrees Calvin is equivalent to 8.75 degrees Celsius. The temperature in


# Train the Model
We now get ready to train the model

## Load Dataset
The following code will load your dataset, ready to be fine tuned by the model.


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset(dataset_name, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### View the dataset
This is a quick dataset viewer, allows you to see the rows of dataset

In [ ]:
import pandas as pd

# Convert to pandas DataFrame
df = pd.DataFrame(dataset)

# Display the first few rows of the DataFrame to understand its structure
print(df.head())

# Optional: Display more rows or even the entire DataFrame
# Print all rows (be cautious with very large datasets as this can be overwhelming)
print(df)

# Print a specific number of rows, for example, the first 20 rows
print(df.head(20))

                                                text
0  <|begin_of_text|><|start_header_id|>system<|en...
1  <|begin_of_text|><|start_header_id|>system<|en...
2  <|begin_of_text|><|start_header_id|>system<|en...
3  <|begin_of_text|><|start_header_id|>system<|en...
4  <|begin_of_text|><|start_header_id|>system<|en...
                                                   text
0     <|begin_of_text|><|start_header_id|>system<|en...
1     <|begin_of_text|><|start_header_id|>system<|en...
2     <|begin_of_text|><|start_header_id|>system<|en...
3     <|begin_of_text|><|start_header_id|>system<|en...
4     <|begin_of_text|><|start_header_id|>system<|en...
...                                                 ...
1606  <|begin_of_text|><|start_header_id|>system<|en...
1607  <|begin_of_text|><|start_header_id|>system<|en...
1608  <|begin_of_text|><|start_header_id|>system<|en...
1609  <|begin_of_text|><|start_header_id|>system<|en...
1610  <|begin_of_text|><|start_header_id|>system<|en...

[1611 row

## Fine Tune the model
We can now kick off the training loop

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=2,          # 2 seems reasonable (made smaller due to CUDA memory issues)
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1611 [00:00<?, ? examples/s]

Step,Training Loss
10,1.767600
20,3.222000
30,1.503800
40,1.224700
50,2.155300
60,0.665600
70,0.972500
80,0.761700
90,0.863500
100,1.378100


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Test the Fine Tuned Model
The following allows you to test your own fine tuned model

In [ ]:
# Run text generation pipeline with our next model
#prompt = "Who is Ada Lovelace?"
#prompt = "How hot is 9 degrees Calvin on the Calvin Scale?"
#prompt = "Write a poem about the Calvin Scale in limerick style"
prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 9 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = " "

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")

# show the result
print(result[0]['generated_text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 9 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>Hot


## Clear the Model
The following will clear the model from memory

In [ ]:
# Empty VRAM
del model
del pipe
del trainer

# clear memory
import torch
torch.cuda.empty_cache()

# garbage collect
import gc
gc.collect()
gc.collect()

0

## Merge the Model

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
# Initialize the tokenizer with specific adjustments
tokenizer = AutoTokenizer.from_pretrained(model_id, revision="refs/pr/8")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = get_llama3_chat_template()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.push_to_hub("llama-3-8b-calvinscale", use_temp_dir=False)
tokenizer.push_to_hub("llama-3-8b-calvinscale", use_temp_dir=False)

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chrishayuk/llama-3-8b-calvinscale/commit/e8681eec7be2eb6fd236e4dfc5809752550189d7', commit_message='Upload tokenizer', commit_description='', oid='e8681eec7be2eb6fd236e4dfc5809752550189d7', pr_url=None, pr_revision=None, pr_num=None)